In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path

data_path = Path('/kaggle/input/h-and-m-personalized-fashion-recommendations/')

In [ ]:
original_data= transactions = pd.read_csv(
    data_path / 'transactions_train.csv',
    dtype={'article_id': str} )

submission = pd.read_csv(data_path / 'sample_submission.csv')

transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions['month'] = transactions['t_dat'].dt.month


In [ ]:
#The span
print(transactions['t_dat'].min())
print(transactions['t_dat'].max())


In [ ]:
#Considering the predicted purchase time is Sep, Only take 6,7,8,9,10,11,12 month into consideration. 
#Using last week of final purchase as validation.
original_transactions = transactions.loc[transactions['month'] >= 6]
transactions = original_transactions.loc[transactions['t_dat']<datetime(2020, 9, 7)]
#valid_transactions = original_data.loc[transactions['t_dat']>=datetime(2020, 9, 7)]


In [ ]:
#Pursue a dict the key is customer_id the value is also a dict of the article and corresponding purchase times.
def create_dict(transactions,purchase_dict):
    for i,x in enumerate(zip(transactions['customer_id'], transactions['article_id'])):
        cust_id, art_id = x
        if cust_id not in purchase_dict:
            purchase_dict[cust_id] = {}
    
        if art_id not in purchase_dict[cust_id]:
            purchase_dict[cust_id][art_id] = 0
    
        purchase_dict[cust_id][art_id] += 1



In [ ]:
#Avoid Out of Memory
n_split_prediction = 1000
purchase_dict = {}
n_chunk = (len(transactions) + n_split_prediction - 1)// n_split_prediction
for i in range(0, len(transactions), n_chunk):
    #print(f"chunk: {i}")
    
    target_transactions = transactions.iloc[i:i+n_chunk]
    create_dict(target_transactions,purchase_dict)
    
    
    

In [ ]:
#Caculate top12 items to make up for the absent position of previous purchase
top12 = list(transactions["article_id"].value_counts().index[:12])

In [ ]:
sub = submission[["customer_id"]]
string_top12 = ' '.join(top12)

#According to the customers submmision provided, to predict items mainly based on the previous purchase.
def generate_prediciton(submission,purchase_dict,prediction_list,top12,string_top12):
    for i, cust_id in enumerate(submission['customer_id'].values):
        if cust_id in purchase_dict:
            l = sorted((purchase_dict[cust_id]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            if len(l)>12:
                s = ' '.join(l[:12])
            else:
                s = ' '.join(l+top12[:(12-len(l))])
        else:
            s = string_top12
        prediction_list.append(s)
    return prediction_list
sub["prediction"]= generate_prediciton(submission,purchase_dict,[],top12,string_top12)
print(sub.head())

 

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
#Evaluation
evaluation = original_data.loc[original_data['t_dat'] >datetime(2020, 9, 7)]
valid_y = list(evaluation["article_id"])
x = evaluation["customer_id"]


def generate_single_prediciton(submission,purchase_dict,prediction_list,top12):
    for i, cust_id in enumerate(submission['customer_id'].values):
        if cust_id in purchase_dict:
            l = sorted((purchase_dict[cust_id]).items(), key=lambda x: x[1], reverse=True)
            l = [y[0] for y in l]
            if len(l)>0:
                s = l[0]
            else:
                s = top12[0]
        else:
            s = top12[0]
        prediction_list.append(s)
    return prediction_list

predicted_y = generate_single_prediciton(evaluation,purchase_dict,[],top12)

correct_count = 0
for i in range(len(predicted_y)):
    if predicted_y[i] == valid_y[i]:
        correct_count += 1
precision = correct_count / len(evaluation)
print(precision)
